In [58]:
%aiida

Loaded AiiDA DB environment - profile name: bmourino.

In [59]:
from __future__ import absolute_import
import os
import glob
import ntpath
from cube import Cube
import pandas as pd
from datetime import datetime

from aiida import load_profile
from aiida.tools import get_kpoints_path
from aiida.orm.querybuilder import QueryBuilder
from aiida.plugins import CalculationFactory, WorkflowFactory
from aiida.orm import StructureData, Dict, load_node, CalcFunctionNode, CalcJobNode, WorkChainNode, BandsData, CifData, Str

In [60]:
def get_base_retrieved_folder(wc_pk): 
    wc = load_node(wc_pk)
    path = load_node(wc.pk).outputs.retrieved._repository._get_base_folder().abspath
    return path

def get_cube_path_smb(wc_pk):
    cubepath = get_base_retrieved_folder(wc_pk)
    cubesmb = '/home/beatriz/.aiida/repository/bmourino/repository/node' + cubepath.split('node')[-1] #can be better - I can make it case-based so it works for all of the backed up folders
    return cubesmb

In [61]:
Cp2kCalculation = CalculationFactory('cp2k')
Cp2kBaseWorkChain = WorkflowFactory('cp2k.base')
Cp2kMultistageWorkChain = WorkflowFactory('lsmo.cp2k_multistage')
GetElectronInjection = WorkflowFactory('photocat_workchains.get_electron_injection')
GetHoleInjection = WorkflowFactory('photocat_workchains.get_hole_injection')
Cp2kPhotoCatWorkChain = WorkflowFactory('photocat_workchains.cp2k_photocat')
#submission_time = datetime(2022, 10, 12, 8,0,0)

In [62]:
qb = QueryBuilder()
qb.append(Group, filters={'label': 'ocofs-conv2e'}, tag='groupe')
qb.append(GetElectronInjection, with_group='groupe', tag='gete')
qb.append(Cp2kBaseWorkChain, with_incoming='gete', project='uuid')
qb.append(Str, with_outgoing='gete', project=['attributes.value'], tag='ms_str')
e_inj = qb.all()

# for i in e_inj:
#     i.append(load_node(i[-1]).caller.called[2].inputs.cifdata.label)
for i in e_inj:
    i.append(load_node(i[-1]).caller.inputs.structure.label)

/home/beatriz/miniconda3/envs/aiida1.6/lib/python3.7/site-packages/aiida/orm/querybuilder.py:271: AiidaEntryPointWarning: Process type 'aiida_cp2k.workchains.base.Cp2kBaseWorkChain' does not correspond to a registered entry. This risks queries to fail once the location of the process class changes. Add an entry point for 'aiida_cp2k.workchains.base.Cp2kBaseWorkChain' to remove this warning.
  "Add an entry point for '{value}' to remove this warning.".format(value=value), AiidaEntryPointWarning


In [63]:
qb = QueryBuilder()
qb.append(Group, filters={'label': 'ocofs-conv2h'}, tag='grouph')
qb.append(GetHoleInjection, with_group='grouph', tag='geth')
qb.append(Cp2kBaseWorkChain, with_incoming='geth', project='uuid')
qb.append(Str, with_outgoing='geth', project=['attributes.value'], tag='ms_str')
h_inj = qb.all()

# for i in h_inj:
#     i.append(load_node(i[-1]).caller.called[2].inputs.cifdata.label)

for i in h_inj:
    i.append(load_node(i[-1]).caller.inputs.structure.label)

In [64]:
h_inj

[['2a6185ce-9e06-46c3-91eb-1437f3eda0ba',
  '9233bdb3-0ebe-47fa-ab57-fc107073df53',
  '20290N2_ddec.cif'],
 ['ddf8970b-e60b-4693-9c8b-354efc485e4d',
  'f508d5ee-0d42-4c41-bd05-2829c3bdbab8',
  '19482N2_ddec.cif'],
 ['bc56681c-0004-43d3-8ee1-3cf27c27fcf9',
  '122054f7-3400-45c2-9cee-7ac8e83b5b20',
  '19103N2_ddec.cif'],
 ['f5fc0f9f-4f63-4eb6-b40c-f316ba9c63b4',
  '3b0e5fc6-10dd-4b27-a295-e5419cc97366',
  '21030N2_ddec.cif'],
 ['f2a89829-dcd7-40f3-a1dc-1ae3ad08697a',
  '3e068c69-8e0c-40a6-8da8-d494101f31b4',
  '15090N2_ddec.cif'],
 ['55f16639-698b-4a6e-ac2f-db2c7ff6262d',
  'c0c148d0-1b6c-455d-a21a-eef2802459f2',
  '20542N2_ddec.cif'],
 ['998746b4-f78b-4a8a-b505-56ebbf00f639',
  'd0119713-de5e-4492-b0f3-a3e8bdc60264',
  '15242N2_ddec.cif'],
 ['f0831473-7702-481f-b406-53cc8d8aed75',
  '87f0a7d1-da76-4193-90a6-4b311ccb826f',
  '16260N2_ddec.cif'],
 ['e8bf72e2-6ae1-4d0b-98c3-75911a0d50ec',
  '16a408e3-90aa-4b16-b6fe-88fb007a828d',
  '16032N2_ddec.cif'],
 ['f5a685e0-8aa9-475e-9ae5-b548a8efd2

In [38]:
ovlp=[]
for i in e_inj[0:]:
    for j in h_inj[0:]:
        if i[-1] == j[-1]:
            try:
                name = i[-1]
                print(name)
                epath = get_cube_path_smb(load_node(i[0]).pk)
                # epath = '/home/beatriz/nas/oehexc7_9_m2_6/node/f9/c9/92e9-b39a-4092-8f9d-20190b82457b/path'
                #epath = '/home/beatriz/nas/oehexc7_9_m2_6/node/ad/09/0f14-e37d-4ebf-aaa6-fcdbd7cc215a/path'
                print(epath)
                ecubes = glob.glob(os.path.join(epath, "aiida-WFN*.cube"))
                hpath = get_cube_path_smb(load_node(j[0]).pk)
                #hpath = '/home/beatriz/nas/oehexc7_9_m2_6/node/95/41/e675-8488-4b73-bb7f-133ba6b8fea1/path'
                #hpath = ''
                print(hpath)
                hcubes = glob.glob(os.path.join(hpath, "aiida-WFN*.cube"))
                a=[]
                c=[]
                for cube in ecubes:
                    b = cube.split('_')[-3]
                    if b in a:
                        c = b
                    else:
                        a.append(b)

                efile = 'aiida-WFN_' + c + '_1-1_0.cube' #was 1
                einj = glob.glob(os.path.join(epath, efile))
                e=[]
                d=[]
                for cube in hcubes:
                    b = cube.split('_')[-3]
                    if b in e:
                        d = b
                    else:
                        e.append(b)

                hfile = 'aiida-WFN_' + d + '_2-1_0.cube'
                hinj = glob.glob(os.path.join(hpath, hfile))      
                print(einj,hinj)
                cubee = Cube(einj[0])
                cubeh = Cube(hinj[0])
                so = cubee.__mul__(cubeh)
                ovlp.append({
                    'name': name,
                    'spatial_overlap': so,
                })
            except Exception as e:
                print(e)
                pass
print('spatial overlap', ovlp)

11030N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/62/f8/60eb-c3b5-4115-9d02-d4a09c4e683e/path
/home/beatriz/.aiida/repository/bmourino/repository/node/70/bb/e4b4-a950-4d31-9d29-a8bf59da173c/path
['/home/beatriz/.aiida/repository/bmourino/repository/node/62/f8/60eb-c3b5-4115-9d02-d4a09c4e683e/path/aiida-WFN_00637_1-1_0.cube'] ['/home/beatriz/.aiida/repository/bmourino/repository/node/70/bb/e4b4-a950-4d31-9d29-a8bf59da173c/path/aiida-WFN_00636_2-1_0.cube']
19570N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/46/1e/5c5f-fafe-4be4-b47a-be50655117db/path
/home/beatriz/.aiida/repository/bmourino/repository/node/27/c8/000a-b5fc-4543-b9e8-7fc5fa95fe21/path
['/home/beatriz/.aiida/repository/bmourino/repository/node/46/1e/5c5f-fafe-4be4-b47a-be50655117db/path/aiida-WFN_01041_1-1_0.cube'] ['/home/beatriz/.aiida/repository/bmourino/repository/node/27/c8/000a-b5fc-4543-b9e8-7fc5fa95fe21/path/aiida-WFN_01040_2-1_0.cube']
20402N2_ddec.cif
/home/beatriz/.ai

In [69]:
done_df = pd.read_csv('../output_data/new/oconv2/ovlp/ovlp.csv')
done = []
for i in done_df['name']:
    done.append(i)

In [70]:
done

['20290N2_ddec.cif',
 '19482N2_ddec.cif',
 '21030N2_ddec.cif',
 '15090N2_ddec.cif',
 '20542N2_ddec.cif',
 '15242N2_ddec.cif',
 '16260N2_ddec.cif',
 '16032N2_ddec.cif',
 '17155N2_ddec.cif',
 '16172N2_ddec.cif',
 '20161N2_ddec.cif',
 '19483N2_ddec.cif',
 '20163N2_ddec.cif',
 '19220N2_ddec.cif',
 '16510N2_ddec.cif',
 '2016bN2_ddec.cif',
 '20167N2_ddec.cif',
 '19341N2_ddec.cif',
 '20510N2_ddec.cif',
 '11001N2_ddec.cif',
 '09000N3_ddec.cif',
 '20000N2_ddec.cif',
 '19560N2_ddec.cif',
 '18140N2_ddec.cif']

In [71]:
ovlp=[]
# missing = ['21091N3_ddec.cif','21001N2_ddec.cif']
for i in e_inj[0:]:
    for j in h_inj[0:]:
        if i[-1] == j[-1]:
            if i[-1] in done:
                continue
            else:
            # if i[-1] in missing:
                try:
                    name = i[-1]
                    print(name)
                    epath = get_cube_path_smb(load_node(i[0]).pk)
                    # epath = '/home/beatriz/nas/oehexc7_9_m2_6/node/f9/c9/92e9-b39a-4092-8f9d-20190b82457b/path'
                    #epath = '/home/beatriz/nas/oehexc7_9_m2_6/node/ad/09/0f14-e37d-4ebf-aaa6-fcdbd7cc215a/path'
                    print(epath)
                    ecubes = glob.glob(os.path.join(epath, "aiida-WFN*.cube"))
                    hpath = get_cube_path_smb(load_node(j[0]).pk)
                    #hpath = '/home/beatriz/nas/oehexc7_9_m2_6/node/95/41/e675-8488-4b73-bb7f-133ba6b8fea1/path'
                    #hpath = ''
                    print(hpath)
                    hcubes = glob.glob(os.path.join(hpath, "aiida-WFN*.cube"))
                    a=[]
                    c=[]
                    for cube in ecubes:
                        b = cube.split('_')[-3]
                        if b in a:
                            c = b
                        else:
                            a.append(b)

                    efile = 'aiida-WFN_' + c + '_1-1_0.cube' #was 1
                    einj = glob.glob(os.path.join(epath, efile))
                    e=[]
                    d=[]
                    for cube in hcubes:
                        b = cube.split('_')[-3]
                        if b in e:
                            d = b
                        else:
                            e.append(b)

                    hfile = 'aiida-WFN_' + d + '_2-1_0.cube'
                    hinj = glob.glob(os.path.join(hpath, hfile))      
                    print(einj,hinj)
                    cubee = Cube(einj[0])
                    cubeh = Cube(hinj[0])
                    so = cubee.__mul__(cubeh)
                    ovlp.append({
                        'name': name,
                        'spatial_overlap': so,
                    })
                except Exception as e:
                    print(e)
                    pass
print('spatial overlap', ovlp)

19103N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/da/a1/ff6b-6d6f-461a-b9d1-242b9359b690/path
Node<25906> does not have an output with link label 'retrieved'
19071N2_ddec.cif
Node<25612> does not have an output with link label 'retrieved'
17050N2_ddec.cif
/home/beatriz/.aiida/repository/bmourino/repository/node/45/b5/5ecb-08df-48ad-95eb-c48b9cae8ed5/path
Node<26174> does not have an output with link label 'retrieved'
spatial overlap []


In [57]:
df = pd.DataFrame(ovlp)
df.to_csv('../output_data/new/oconv2/ovlp/ovlp2.csv',index=False)

In [422]:
q = load_node(h_inj[2][0])
q.outputs.retrieved._repository._get_base_folder().abspath

'/home/beatriz/aiida1/.aiida/repository/Beatriz/repository/node/18/0f/54fa-5fbb-463e-b39e-d8e1d1036345/path'

In [ ]:
q = load_node(e_inj[2][0])
q.outputs.retrieved._repository._get_base_folder().abspath

In [217]:
h_inj

[['d09f0b3d-40d2-4956-8010-cca566b65b5e',
  'a80fbaef-8665-442a-bd36-7e90cfcc07e0',
  '13075N2_ddec.cif'],
 ['b4b55e86-085e-4736-8de4-805e50861c45',
  'f747155d-aaa9-42b0-95fe-8108f2f02359',
  '13011N2_ddec.cif'],
 ['ea6984bb-fa3e-4481-880e-12b6742b74c4',
  '63dbaeea-734f-4c4d-a10a-be0fe2d7b431',
  '13040N3_ddec.cif'],
 ['5a56b232-8ccc-4645-b792-8313c00532d9',
  'a3e5ca9c-5df3-400b-ad02-b4d850ba64a9',
  '2016aN2_ddec.cif'],
 ['e9a6b56c-97e0-464e-b0d6-5db8d6e9e798',
  'd9a7e5b7-39bc-4d71-a971-f8445bd68606',
  '13051N2_ddec.cif'],
 ['b7e44825-ff8c-42cc-ae64-e2ea2e894a42',
  '87196db3-6794-49e0-b05f-ecbe77bf40c5',
  '12030N2_ddec.cif'],
 ['d57f861e-5ebe-466d-b13f-be67b84f74ee',
  '61430ce7-5fdf-4947-a768-161e99ff65e0',
  '13060N2_ddec.cif'],
 ['412ffcee-a784-4255-98a8-41c29b6908ed',
  'd2deeb96-fcc6-4815-82f6-6d6cc4a67bec',
  '13010N2_ddec.cif'],
 ['1b7f7522-9f76-464c-8b23-990bd7f64b9c',
  '0507a3a2-cdf3-4315-91c1-bd5000f7add3',
  '10010N2_ddec.cif'],
 ['538f689d-9969-485a-bf27-1a969d4d8e

In [79]:
#ovlp = []
name = e_inj[2][-1]
#name = '2016aN2_ddec.cif'
print(name)
#epath = get_cube_path_smb(load_node(e_inj[-1][0]).pk)
epath = '/home/beatriz/nas/rmqeh/17151N2_ddec.cif'
ecubes = glob.glob(os.path.join(epath, "aiida-WFN*.cube"))
print(epath)
#hpath = '/home/beatriz/nas/oehexc7_9_m2_6/node/44/59/d0b3-daac-4182-85f1-644754ba30a9/path'
hpath = get_cube_path_smb(load_node(h_inj[2][0]).pk)
#hpath = '/home/beatriz/nas/rmqeh/16330N2_ddec.cif'
print(hpath)
hcubes = glob.glob(os.path.join(hpath, "aiida-WFN*.cube"))
a=[]
c=[]
for cube in ecubes:
    b = cube.split('_')[-3]
    if b in a:
        c = b
    else:
        a.append(b)

efile = 'aiida-WFN_' + c + '_1-1_0.cube'
einj = glob.glob(os.path.join(epath, efile))
e=[]
d=[]
for cube in hcubes:
    b = cube.split('_')[-3]
    if b in e:
        d = b
    else:
        e.append(b)

hfile = 'aiida-WFN_' + d + '_2-1_0.cube'
hinj = glob.glob(os.path.join(hpath, hfile))      
print(einj,hinj)
cubee = Cube(einj[0])
cubeh = Cube(hinj[0])
so = cubee.__mul__(cubeh)
ovlp.append({
    'name': name,
    'spatial_overlap': so,
})
print('spatial overlap', ovlp)

17151N2_ddec.cif
/home/beatriz/nas/rmqeh/17151N2_ddec.cif
/home/beatriz/nas/oehexc7_9_m2_6/node/18/0f/54fa-5fbb-463e-b39e-d8e1d1036345/path
['/home/beatriz/nas/rmqeh/17151N2_ddec.cif/aiida-WFN_00977_1-1_0.cube'] ['/home/beatriz/nas/oehexc7_9_m2_6/node/18/0f/54fa-5fbb-463e-b39e-d8e1d1036345/path/aiida-WFN_00976_2-1_0.cube']
spatial overlap [{'name': '18060N2_ddec.cif', 'spatial_overlap': 0.6298251748085022}, {'name': '17070N2_ddec.cif', 'spatial_overlap': 0.5986355543136597}, {'name': '17151N2_ddec.cif', 'spatial_overlap': 0.6714895367622375}, {'name': '17140N2_ddec.cif', 'spatial_overlap': 0.6192883849143982}, {'name': '17090N3_ddec.cif', 'spatial_overlap': 0.5890432596206665}, {'name': '17240N3_ddec.cif', 'spatial_overlap': 0.5585640668869019}, {'name': '17155N2_ddec.cif', 'spatial_overlap': 0.4693623185157776}, {'name': '17230N2_ddec.cif', 'spatial_overlap': 0.5905717015266418}, {'name': '18031N2_ddec.cif', 'spatial_overlap': 0.5000246167182922}, {'name': '17060N2_ddec.cif', 'spatial

In [63]:
ovlp

[{'name': '16311N2_ddec.cif', 'spatial_overlap': 0.20475254952907562},
 {'name': '16381N2_ddec.cif', 'spatial_overlap': 0.7452120184898376},
 {'name': '16220N2_ddec.cif', 'spatial_overlap': 0.6040955185890198},
 {'name': '16331N2_ddec.cif', 'spatial_overlap': 0.6669694781303406},
 {'name': '16340N2_ddec.cif', 'spatial_overlap': 0.47119852900505066},
 {'name': '16300N2_ddec.cif', 'spatial_overlap': 0.7416027188301086},
 {'name': '16190N2_ddec.cif', 'spatial_overlap': 0.34230223298072815},
 {'name': '16320N2_ddec.cif', 'spatial_overlap': 0.597741961479187},
 {'name': '16430N2_ddec.cif', 'spatial_overlap': 0.4871460199356079},
 {'name': '16172N2_ddec.cif', 'spatial_overlap': 0.6358146071434021},
 {'name': '16380N2_ddec.cif', 'spatial_overlap': 0.5779324173927307},
 {'name': '16330N2_ddec.cif', 'spatial_overlap': 0.5545110702514648},
 {'name': '16210N2_ddec.cif', 'spatial_overlap': 0.40234389901161194}]

In [ ]:
#try: filters={'and': [{'attributes.process_state':{'==':'finished'}}]},